In [1]:
import numpy as np
import os
from numpy import asarray
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten,MaxPooling2D,Conv2D,Dropout

In [4]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [2]:
DIR= "/content/drive/MyDrive/asl-dataset"
IMG_SHAPE=(200,200,3)
IMG_SIZE=200
NUM_OF_CLASSES=29
CATEGORIES=["A","B","C","D","del","E","F","G",
            "H","I","J","K","L","M","N","nothing",
            "O","P","Q","R","S","space","T","U",
            "V","W","X","Y","Z"]

In [3]:
dataset_train=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(200,200),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="training"
)

Found 10854 files belonging to 29 classes.
Using 7273 files for training.


In [4]:
dataset_val=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(200,200),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="validation"
)

Found 10854 files belonging to 29 classes.
Using 3581 files for validation.


In [5]:
model=tf.keras.models.Sequential()
model.add(Conv2D(512, (3, 3),input_shape=IMG_SHAPE,activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(256, (3, 3),activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3),activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation=tf.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(NUM_OF_CLASSES,activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 512)     14336     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 512)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 256)       1179904   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       295040    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        7

In [6]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(dataset_train,validation_data = dataset_val, epochs=10,verbose=2)

Epoch 1/10
228/228 - 137s - loss: 4.6503 - accuracy: 0.1096 - val_loss: 2.7898 - val_accuracy: 0.2159
Epoch 2/10
228/228 - 99s - loss: 2.2493 - accuracy: 0.3569 - val_loss: 1.9098 - val_accuracy: 0.4538
Epoch 3/10
228/228 - 100s - loss: 1.3159 - accuracy: 0.5959 - val_loss: 1.3197 - val_accuracy: 0.6107
Epoch 4/10
228/228 - 100s - loss: 0.8190 - accuracy: 0.7419 - val_loss: 1.0315 - val_accuracy: 0.7018
Epoch 5/10
228/228 - 100s - loss: 0.4770 - accuracy: 0.8375 - val_loss: 1.0788 - val_accuracy: 0.7328
Epoch 6/10
228/228 - 101s - loss: 0.3403 - accuracy: 0.8890 - val_loss: 0.9707 - val_accuracy: 0.7716
Epoch 7/10
228/228 - 101s - loss: 0.2626 - accuracy: 0.9142 - val_loss: 0.8776 - val_accuracy: 0.8040
Epoch 8/10
228/228 - 100s - loss: 0.2131 - accuracy: 0.9302 - val_loss: 1.0170 - val_accuracy: 0.7992
Epoch 9/10
228/228 - 100s - loss: 0.1977 - accuracy: 0.9369 - val_loss: 0.8782 - val_accuracy: 0.8174
Epoch 10/10
228/228 - 100s - loss: 0.1539 - accuracy: 0.9501 - val_loss: 1.1033 - v

In [7]:
model.save("asl_classifier.model")

INFO:tensorflow:Assets written to: asl_classifier.model/assets


In [12]:
base_model=tf.keras.applications.VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(200, 200, 3)
)

base_model.trainable=False

In [13]:
model = tf.keras.models.Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(NUM_OF_CLASSES,activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 29)                534557    
Total params: 15,249,245
Trainable params: 534,557
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.fit(dataset_train,validation_data = dataset_val, epochs=10,verbose=2)

Epoch 1/10
228/228 - 56s - loss: 6.5195 - accuracy: 0.7366 - val_loss: 2.2999 - val_accuracy: 0.8780
Epoch 2/10
228/228 - 54s - loss: 1.2680 - accuracy: 0.9295 - val_loss: 2.5961 - val_accuracy: 0.8852
Epoch 3/10
228/228 - 55s - loss: 0.8899 - accuracy: 0.9517 - val_loss: 2.1571 - val_accuracy: 0.9126
Epoch 4/10
228/228 - 55s - loss: 0.6769 - accuracy: 0.9654 - val_loss: 2.9961 - val_accuracy: 0.8992
Epoch 5/10
228/228 - 55s - loss: 0.6151 - accuracy: 0.9711 - val_loss: 1.5868 - val_accuracy: 0.9391
Epoch 6/10
228/228 - 56s - loss: 0.7331 - accuracy: 0.9682 - val_loss: 1.7050 - val_accuracy: 0.9408
Epoch 7/10
228/228 - 56s - loss: 0.4892 - accuracy: 0.9792 - val_loss: 1.8143 - val_accuracy: 0.9347
Epoch 8/10
228/228 - 55s - loss: 0.8597 - accuracy: 0.9671 - val_loss: 2.1685 - val_accuracy: 0.9450
Epoch 9/10
228/228 - 55s - loss: 0.5956 - accuracy: 0.9787 - val_loss: 1.5065 - val_accuracy: 0.9542
Epoch 10/10
228/228 - 56s - loss: 0.3000 - accuracy: 0.9839 - val_loss: 2.3469 - val_accura